# DAY 2: RAG - Full Pipeline Example

### What this file does:
Demonstrates a complete Retrieval-Augmented Generation (RAG) solution: Chunk, embed, and store data in Oracle DB. Retrieval: Embed query and perform similarity search. Augmentation: Add retrieved chunks to prompt. Generation: Get response from LLM with citations.

**Documentation to reference:**
- OCI GenAI Embeddings: https://docs.oracle.com/en-us/iaas/Content/generative-ai/pretrained-models.htm#embed-models
- OCI GenAI Chat: https://docs.oracle.com/en-us/iaas/Content/generative-ai/chat-models.htm
- Oracle DB Vectors: https://docs.oracle.com/en/database/oracle/oracle-database/23/vecse/
- OCI Python SDK: https://github.com/oracle/oci-python-sdk/tree/master/src/oci/generative_ai_inference

**Relevant slack channels:**
- #generative-ai-users or #igiu-innovation-lab: *For questions*
- #igiu-ai-learning: *For errors*

**Env setup:**
- sandbox.yaml: Needs "oci" (configFile, profile, compartment, bucket) and "db" (tablePrefix, walletPath, username, password, dsn, walletPass) sections.
- .env: Load environment variables if needed.
- Download wallet for DB access.
- configure cwd for jupyter match your workspace python code: 
    -  vscode menu -> Settings > Extensions > Jupyter > Notebook File Root
    -  change from `${fileDirname}` to `${workspaceFolder}`


**How to run in notebook:**
- Ensure dependencies installed (uv sync).
- Update sandbox.yaml with DB details and wallet path.
- Run cells in order. Requires Oracle DB access.

In [ ]:
# Import libraries
from oci.generative_ai_inference import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import OnDemandServingMode, EmbedTextDetails, CohereChatRequest, ChatDetails
import oracledb
import array
import oci
import os
from dotenv import load_dotenv
from envyaml import EnvYAML

In [ ]:
# Constants
SANDBOX_CONFIG_FILE = "sandbox.yaml"
load_dotenv()
EMBED_MODEL = "cohere.embed-multilingual-v3.0"
LLM_MODEL = "cohere.command-a-03-2025"
llm_service_endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

In [ ]:
# Mock chunks (in practice, parse real documents)
chunks = [
    "Baseball is a great game ",
    "baseball games typically last 9 innings",
    "Baseball game can finish in about 2 hours",
    "Indias favorite pastime is cricket",
    "England's favorite pastime is football",
    "Football is called soccer in America",
    "baseball is americas favorite pastime sport"
]

# Metadata for chunks
chunk_source = [
    {"chapter": "Baseball", "question": "1"},
    {"chapter": "Baseball", "question": "2"},
    {"chapter": "Baseball", "question": "3"},
    {"chapter": "Cricket", "question": "1"},
    {"chapter": "Football", "question": "1"},
    {"chapter": "Football", "question": "2"},
    {"chapter": "Baseball", "question": "4"}
]

### Step 1: Load config and initialize OCI clients for embeddings and chat

In [ ]:
# Load config
scfg = EnvYAML(SANDBOX_CONFIG_FILE)
if scfg is None or "oci" not in scfg or "bucket" not in scfg:
    raise RuntimeError("Invalid sandbox configuration.")

config = oci.config.from_file(os.path.expanduser(scfg["oci"]["configFile"]), scfg["oci"]["profile"])
compartmentId = scfg["oci"]["compartment"]
tablename_prefix = scfg["db"]["tablePrefix"]
wallet = os.path.expanduser(scfg["db"]["walletPath"])

In [ ]:
# Connect to DB
db = oracledb.connect(
    config_dir=scfg["db"]["walletPath"],
    user=scfg["db"]["username"],
    password=scfg["db"]["password"],
    dsn=scfg["db"]["dsn"],
    wallet_location=scfg["db"]["walletPath"],
    wallet_password=scfg["db"]["walletPass"]
)
cursor = db.cursor()

In [ ]:
# Initialize LLM client
llm_client = GenerativeAiInferenceClient(
    config=config,
    service_endpoint=llm_service_endpoint,
    retry_strategy=oci.retry.NoneRetryStrategy(),
    timeout=(10, 240)
)

### Step 2: Create Vector Table in DB

In [ ]:
# Set up the Oracle DB table to store text chunks, embeddings, and metadata.
# Create table for embeddings
sql = [
    f"""drop table if exists {tablename_prefix}_embedding purge""",
    f"""
    create table {tablename_prefix}_embedding (
        id number,
        text varchar2(4000),
        vec vector,
        chapter varchar2(100),
        section integer,
        primary key (id)
    )"""
]

for s in sql:
    cursor.execute(s)

db.commit()

### Step 3: Generate Embeddings and Store in DB

In [ ]:
# Embed the text chunks and insert them into the vector table.
# Build embedding payload
embed_text_detail = EmbedTextDetails()
embed_text_detail.serving_mode = OnDemandServingMode(model_id=EMBED_MODEL)
embed_text_detail.truncate = embed_text_detail.TRUNCATE_END
embed_text_detail.input_type = EmbedTextDetails.INPUT_TYPE_SEARCH_DOCUMENT
embed_text_detail.compartment_id = compartmentId
embed_text_detail.inputs = chunks

# Generate embeddings
response = llm_client.embed_text(embed_text_detail)
embeddings = response.data.embeddings

# Insert into DB
for i in range(len(embeddings)):
    cursor.execute(
        f"insert into {tablename_prefix}_embedding values (:1, :2, :3, :4, :5)",
        [i, chunks[i], array.array("f", embeddings[i]), chunk_source[i]["chapter"], chunk_source[i]["question"]]
    )
    print(f"inserted {i}-{chunks[i]}")

db.commit()

### Step 4: Verify Stored Data

In [ ]:
# Check the contents of the vector table.
# Query stored chunks
cursor.execute(f"select id,text from {tablename_prefix}_embedding")
for row in cursor:
    print(f"{row[0]}:{row[1]}")

### Step 5: Interactive Query and RAG

In [ ]:
# Embed user query, retrieve similar chunks, augment prompt, and generate response.
# User query
query = input("Ask a question: ").strip().lower()
q = [query]

# Embed query
embed_text_detail.inputs = q
embed_text_detail.input_type = EmbedTextDetails.INPUT_TYPE_SEARCH_QUERY
response = llm_client.embed_text(embed_text_detail)
vec = array.array("f", response.data.embeddings[0])

# Similarity search (COSINE; try DOT or EUCLIDEAN)
cursor.execute(
    f"""
    select id,text, vector_distance(vec, :1, COSINE) d, chapter,section
    from {tablename_prefix}_embedding
    order by d
    fetch first 3 rows only
    """,
    [vec]
)

rows = []
for row in cursor:
    r = [row[0], row[1], row[2], f"chapter:[{row[3]}]_section:[{row[4]}]"]
    print(r)
    rows.append(r)

print(rows)

# Optional: Rerank here (see cohere rerank example)

# Augment prompt
docs = []
for chunk in rows:
    doc = {
        "id": chunk[3],
        "snippet": chunk[1]
    }
    docs.append(doc)

# Build chat request
cohere_chat_request = CohereChatRequest()
# cohere_chat_request.preamble_override = "answer only from selected docs"
cohere_chat_request.is_stream = False
cohere_chat_request.max_tokens = 500
cohere_chat_request.temperature = 0.75
cohere_chat_request.top_p = 0.7
cohere_chat_request.frequency_penalty = 1.0
cohere_chat_request.documents = docs

chat_detail = ChatDetails()
chat_detail.serving_mode = OnDemandServingMode(model_id=LLM_MODEL)
chat_detail.compartment_id = compartmentId
chat_detail.chat_request = cohere_chat_request

# Generate response
cohere_chat_request.message = query
llm_response = llm_client.chat(chat_detail)

# Print results
print("************************** Chat Result **************************")
print(query)
print(llm_response.data.chat_response.text)
print("************************** Citations **************************")
print(llm_response.data.chat_response.citations)

### Step 6: Cleanup

In [ ]:
# Close DB connections.
# Close connections
cursor.close()
db.close()

## Experimentation Section
Good for experimentation:
- Chunking: Replace mock chunks with parsed PDFs (use PyPDFLoader).
- Similarity: Change to DOT_PRODUCT or EUCLIDEAN in vector_distance.
- Retrieval: Adjust FETCH FIRST (e.g., top 5), add distance filter.
- Augmentation: Add reranking before generating.
- Generation: Tune temperature (0.3 factual vs. 1.0 creative).
- Try different EMBED_MODEL or LLM_MODEL.

## Practice Exercises
1. **Talk-to-Document**: Build an app to upload PDFs, chunk, embed, and query.
2. **Text vs. PDF**: Compare using plain text chunks vs. PDF parsing.
3. **Reranking**: Integrate Cohere rerank to reorder retrieved chunks.
4. **Discussion**: What chunk size works best? How does similarity metric affect results?

For help, reach out in #igiu-innovation-lab or #igiu-ai-learning.